In [1]:
from glob import glob
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import json
import os
import mp
import re
import subprocess
import soundfile as sf

In [2]:
files = glob('dialects_processed/**/*/*.json', recursive = True)

len(files)

172101

In [3]:
rejected = [
    'terima kasih kerana menonton',
    'terima kasih',
    'thank you for watching',
]

In [4]:
threshold = -18

def new_path(f):
    f = f.replace('.mp3', '.alignment')
    f = f.replace('_processed/', '_processed_alignment/')
    return f

def new_path_lang(f):
    f = f.replace('.mp3', '.language')
    f = f.replace('_processed/', '_processed_language/')
    return f

def chunk(alignment, reject = threshold, minimum_length = 2.0):
    alls, temp = [], []
    for a in alignment:
        if a['score'] <= reject:
            if len(temp):
                temp[-1]['end'] = a['start']
                if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
                    alls.append(temp)
                temp = []
        else:
            temp.append(a)
            
    if len(temp):
        if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
            alls.append(temp)
    return alls
        
def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string.lower()

def detect_extra(word):
    word = clean(word)
    return word in [
        'eh', 'ehh', 'oh', 'ohh', 'uhm', 'uhmm',
        'um', 'em', 'emm', 'ah', 'ha', 'ok', 'okay',
        'so', 'yes', 'no', 'ah', 'aa', 'so', 'uh', 'um', 'eh', 'ha', 'oh', 'ye', 'haa',
        'oi', 'ya', 'leh', 'lah', 'haiya', 'hoi', 'haha', 'hahaha',
        'then', 'it s'
    ]

from collections import defaultdict

def generate_trigrams(text):
    words = text.split()
    return list(zip(words, words[1:], words[2:]))

def skip_trigrams(text):
    trigrams = generate_trigrams(text)
    count = defaultdict(int)
    total = 0
    for t in trigrams:
        count[''.join(t)] += 1
        total += 1
    if len(count.keys()) < 3:
        return True
    for k, v in count.items():
        if (v / total) > 0.2:
            return True
    return False

In [5]:
!rm -rf dialects-chunk
!mkdir dialects-chunk

In [6]:
def loop(files):
    files, _ = files
    data = []
    for file in tqdm(files):
        folder = os.path.split(file)[0]
        folder_folder = os.path.split(folder)[1]
        filename = file.replace('.json', '')

        try:
            with open(file) as fopen:
                d = json.load(fopen)
        except:
            continue

        for no, obj in enumerate(d):
            text = obj["text"].strip()
            
            rt_ = re.sub('[^a-z ]+', '', text.lower()).strip()
            if any([s == rt_ for s in rejected]):
                continue
            
            try:
                dense = CountVectorizer(ngram_range = (3,3)).fit_transform([text]).todense()
                repeat = (dense > 3).sum() >= 1
                if repeat:
                    continue
            except:
                continue
            
            audio_path = os.path.join(folder, f'{folder_folder}_{no}.mp3')
            
            if not os.path.exists(audio_path):
                continue
                
            align_path = new_path(audio_path)
            
            if not os.path.exists(align_path):
                continue
                
            with open(align_path) as fopen:
                align = json.load(fopen)
                
            for a in align:
                if detect_extra(a['text']):
                    a['score'] = 0.0
                
            scores = [a for a in align if a['score'] <= threshold]
            if not len(scores):
                continue
            
            chunks = chunk(align)
            if len(chunks):
                y, sr = sf.read(audio_path)
                for no, c in enumerate(chunks):
                    if len(c) == len(align):
                        continue
                    try:
                        
                        words = [c_['text'] for c_ in c if len(c_['text']) <= 1]
                        if (len(words) / len(c)) > 0.5:
                            print(c)
                            continue
                        
                        skip = False
                        
                        for c_ in c:
                            if (c_['end'] - c_['start']) >= 2:
                                skip = True
                                break
                        if skip:
                            continue
                        
                        for i in range(1, len(c), 1):
                            if (c[i]['start'] - c[i - 1]['end']) >= 2:
                                skip = True
                                break
                        if skip:
                            continue
                            
                        t = ' '.join([c_['text'] for c_ in c])
                        start = c[0]['start']
                        end = c[-1]['end']
                        
                        a = audio_path.replace('/', '_').replace('.mp3', '') 
                        a = os.path.join('dialects-chunk', f'{a}_{no}.mp3')
                        if not os.path.exists(a):
                            sf.write(a, y[int(sr * start): int(sr * end)], sr)

                        data.append({
                            'audio': a,
                            'transcription': t,
                        })
                    except:
                        pass
    
    return data

In [7]:
d = loop((files[-1:], 0))

100%|███████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.17it/s]


In [8]:
d[-3]

{'audio': 'dialects-chunk/dialects_processed_Bila ckp elok2 kau buat x kisah, jadi, org tu pun perli lah sbb dh x tahan. #DuluDollahDaebak [tAX6gay5Sco]_Bila ckp elok2 kau buat x kisah, jadi, org tu pun perli lah sbb dh x tahan. #DuluDollahDaebak [tAX6gay5Sco]_1_1.mp3',
 'transcription': 'sama je. Tak! Dia macam ni!'}

In [9]:
import IPython.display as ipd
ipd.Audio(d[-3]['audio'])

In [ ]:
data = mp.multiprocessing(files, loop, cores = 20)

In [11]:
len(data)

1541838

In [12]:
from collections import defaultdict

uniques = defaultdict(int)
for d in tqdm(data):
    uniques[d['audio'].split('_processed')[0]] += 1
    
uniques

100%|██████████████████████████████████████████████████████████████████████████| 1541838/1541838 [00:00<00:00, 3505918.86it/s]


defaultdict(int, {'dialects-chunk/dialects': 1541838})

In [13]:
import pandas as pd

df = pd.DataFrame(data)
df.head()

,audio,transcription
0,dialects-chunk/dialects_processed_KILAUAN EMAS...,"Kaya ni, kak ni tau. Jadi, Mama yakin tak Mama..."
1,dialects-chunk/dialects_processed_KILAUAN EMAS...,"Ya, Puan Sharifah, tahniah."
2,dialects-chunk/dialects_processed_KILAUAN EMAS...,kasih banyak-banyak. Terima kasih Encik Nizam ...
3,dialects-chunk/dialects_processed_Sepak Takraw...,"Baik, time out yang diminta, eh, ketika ini, k..."
4,dialects-chunk/dialects_processed_Sepak Takraw...,"untuk time out, eh, yang diminta sementara itu..."


In [14]:
df.to_parquet('verify-text-chunk-dialects.parquet')

In [15]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="verify-text-chunk-dialects.parquet",
    path_in_repo="data/dialects_chunk-00000-of-00001.parquet",
    repo_id="mesolitica/Malaysian-Voice-Conversion",
    repo_type="dataset",
)


verify-text-chunk-dialects.parquet:   0%|          | 0.00/120M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion/commit/a51aa89fdcc1e621f5a2ae6429eaed84b503a289', commit_message='Upload data/dialects_chunk-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='a51aa89fdcc1e621f5a2ae6429eaed84b503a289', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-Voice-Conversion'), pr_revision=None, pr_num=None)